In [1]:
#En este notebook vamos a juntar todos los features calculados hasta ahora para ver si funcionan bien juntos

In [119]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
%matplotlib inline

In [121]:
df_palabras = pd.read_csv('../features/palabras_clave.csv')
df_word2vec = pd.read_csv('../features/word2vec.csv')
df_features_basicas = pd.read_csv('../features/features_basicas_texto.csv',index_col = 'id')
df_keyword = pd.read_csv('../features/features_basicas_keyword.csv',index_col = 'id')
df_location = pd.read_csv('../features/features_location.csv',index_col = 'id')
df_link = pd.read_csv('../features/links_and_hashtags.csv',index_col = 'id')

df_features = df_palabras.join(df_word2vec,how = 'inner')
df_features = df_features.join(df_keyword,how = 'inner')
df_features = df_features.join(df_location,how = 'inner')
df_features = df_features.join(df_link,how = 'inner')
df_features = df_features.join(df_features_basicas,how = 'inner')
df_features = df_features.drop(['mencion_divina','contiene_risa','contiene_hora_especifica',
                                'similitud_debris','similitud_derailment','similitud_wreckage',
                                'similitud_outbreak','similitud_typhoon','similitud_bombing'],axis = 1)
df_features

,mencion_personal,Uso_infinitivos,similitud_con_I,cercania_bombing,cercania_debris,cercania_derailment,cercania_wreckage,cercania_outbreak,has_keyword,keyword_count,...,word_count_above_mean,unique_words_count_above_mean,char_count,char_count_above_mean,avg_word_length,subjectivity,polarity,neg,neu,pos
0,True,False,0.714178,False,False,False,False,False,0,87,...,0,0,24,0,6.000000,1.000000,-0.7003,1,0,0
1,False,False,0.714183,False,True,False,False,False,0,87,...,0,0,38,0,5.428571,0.000000,0.2732,0,0,1
2,False,False,0.908953,False,False,False,False,False,0,87,...,0,0,46,0,6.571429,0.550000,0.4404,0,0,1
3,False,False,0.874617,False,False,False,False,False,0,87,...,1,1,52,1,5.200000,0.000000,-0.6159,1,0,0
4,False,False,0.799880,False,False,False,False,False,0,87,...,0,0,31,0,4.428571,0.400000,-0.3400,1,0,0
5,False,False,0.799843,False,False,False,False,False,0,87,...,1,0,78,1,7.090909,0.400000,0.0000,0,1,0
6,False,True,0.916451,False,False,False,False,False,0,87,...,0,0,53,1,6.625000,0.000000,0.0000,0,1,0
7,True,False,0.833233,False,False,False,False,False,0,87,...,1,1,47,0,4.700000,0.000000,0.0000,0,1,0
8,False,True,0.999914,False,False,False,False,False,0,87,...,1,1,78,1,5.200000,0.237500,-0.3400,1,0,0
9,True,True,0.999818,False,False,False,False,False,0,87,...,0,0,34,0,8.500000,0.000000,0.0000,0,1,0


In [122]:
df_train = pd.read_csv('../dataset/train.csv', index_col='id')
df_test = pd.read_csv('../dataset/test.csv', index_col='id')

features_cols = df_features.columns.tolist()
df_train = df_train.join(df_features, how='inner')

X = df_train[features_cols]
Y = df_train['target']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=17)

In [123]:
from sklearn.model_selection import GridSearchCV
param_grid = { 
    'n_estimators': [200, 500,1000],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
rfc=RandomForestClassifier(random_state=42)
# Instantiate the grid search model
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'

In [124]:
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'auto',
 'n_estimators': 500}

In [125]:
clf1 = RandomForestClassifier(criterion='gini',
 max_depth= 8,
 max_features= 'auto',
 n_estimators= 500)

In [126]:
def try_classifier(classifier):
    classifier.fit(X_train, y_train.values.ravel())
    predictions = classifier.predict(X_test)
    predictions_with_prob = classifier.predict_proba(X_test)[:,1]
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    print(f'Verdaderos Negativos: {tn}')
    print(f'Falsos Negativos: {fn}')
    print(f'Verdaderos Positivos: {tp}')
    print(f'Falsos Positivos: {fp}')
    print()
    print(f'accuracy score: {accuracy_score(y_test, predictions)}')
    print(f'roc auc score: {roc_auc_score(y_test,  predictions_with_prob)}')
    print(f'f1 score: {f1_score(y_test,  predictions)}')
    print()
    return predictions
predictions = try_classifier(clf1)

Verdaderos Negativos: 724
Falsos Negativos: 253
Verdaderos Positivos: 414
Falsos Positivos: 132

accuracy score: 0.747209455022981
roc auc score: 0.8142330703806975
f1 score: 0.6826051112943117



In [127]:
df_submit = df_test.join(df_features, how='inner')[features_cols]
df_submit.shape

(3263, 39)

In [128]:
kaggle_pred = clf1.predict(df_submit)
len(kaggle_pred)

3263

In [129]:
df_submit['target'] = kaggle_pred

In [130]:
df_submit.reset_index(inplace = True)
df_submit.rename(columns={'index':'id'}, inplace=True)
df_submit =df_submit[['id','target']]

In [131]:
df_submit[['id', 'target']].to_csv('../submits/all_features.csv',index=False)